导包

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pymysql

通过url爬取部门、日期、浏览量、正文

In [2]:
def parsecontent(newsurl):
#     处理空url
    if newsurl == '':
        url = "http://www.cuc.edu.cn/news/2021/0309/c1901a178590/page.htm"
    else:
        url = newsurl
    
    r = requests.get(url)
    rt = r.content
    rh = str(rt,"utf-8")
    soup = bs(rt,"html.parser")
#     print(soup)

#     h1 = soup.find_all("h1")
#     print(h1)

#     "class":"arti-name"中包含部门、日期、浏览量
    info = soup.find_all("span",attrs={"class":"arti-name"})
#     print(info)
    infotext = info[0].get_text()
#     print(infotext)

#     爬取部门
#     +3为了除去“来源：”占用的3个字符,截至到20，20为年份开头
    department = infotext[infotext.find('来源：')+3:infotext.find('20')]
#     print(department)

#     正则匹配
    rn = re.compile('[\u4e00-\u9fa5]+')
    dep = rn.findall(department)
#     print(dep[0])
    
#     爬取时间
    date = infotext[infotext.find('20'):infotext.find('20')+10]
#     print(date)

#     由于浏览量的位数不确定，故不采用此方法
#     count = infotext[infotext.find('浏览量：')+4:infotext.find('浏览量：')+6]
#     print(count)

#     爬取浏览量
    countinfo = soup.find_all("span",attrs={"class":"WP_VisitCount"})
    count = countinfo[0].get_text()
#     print(count)

#     爬取内容
    contentinfo = soup.find_all("div",attrs={"class":"wp_articlecontent"})
    content = contentinfo[0].get_text()
#     print(content)
    
    return dep[0],date,count,content

连接数据库并保存信息

In [3]:
def saveinfo(title,newsurl,department,date,count,content):
#     连接数据库
    db = pymysql.connect(host="localhost",user="zouzh",passwd="zouzihan0706",db="class")
    cursor = db.cursor()
#     cursor.execute("INSERT INTO cucnews(title,newsurl,department,date,count,content)VALUES(%s,%s,%s,%s,%s,%s)",(title,newsurl,department,date,count,content))
#     db.commit()
    try:
        cursor.execute("INSERT INTO cucnews(title,newsurl,department,date,count,content)VALUES(%s,%s,%s,%s,%s,%s)",(title,newsurl,department,date,count,content))
        db.commit()
        i = i+1
#         print("db-ok")
    except:
#         print("db-error")
        db.rollback()
    db.close()

主函数

In [4]:
urlbegin = "http://www.cuc.edu.cn/news/1901/list"
# k用于计数
k = 0
for i in range(1,581):
    url = urlbegin+str(i)+".htm"
    r = requests.get(url)
    rt = r.content
    rh = str(rt,"utf-8")
    soup = bs(rh,"html.parser")
    tit = soup.find_all("h3",attrs={'class','tit'})
    for t in tit:
        newsurl = t.find_all('a')
        urllen = str(newsurl[0]).find("target")
        title = t.get_text()
        newsurl = 'http://www.cuc.edu.cn' + (str(newsurl[0])[9:urllen-2])
#         对每个页面内容进行爬取
        department,date,count,content = parsecontent(newsurl)
#         测试
#         m = parsecontent(newsurl)
#         print(department)
        saveinfo(title,newsurl,department,date,count,content)
        k = k+1
        print(k)
#         print(title)
#         print(newsurl)
#         print(department)
#         print(date)
#         print(count)
#         print(content)
print("successful!")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))